In [1]:
%matplotlib inline
#basic python packages for plotting and array management
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
#import statsmodels.api as sm
from datetime import datetime

#for data import
import pandas as pd
from __future__ import division
import os
import sys
from importlib import reload
reload(sys)

#Preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA

#Classifiers
from sklearn.linear_model import LogisticRegression, RidgeClassifier, RidgeClassifierCV, PassiveAggressiveClassifier, Perceptron, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neural_network import MLPClassifier

#Ensemble methods and crossvalidation
from sklearn.model_selection import GridSearchCV, cross_val_predict, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier,RandomForestClassifier, VotingClassifier

#PostProcessing
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

import time

In [2]:
customers = pd.read_csv("customers5.csv")
products = pd.read_csv("products.csv")
X_train = pd.read_csv("X_train.csv")
X_test   = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")

# CUSTOMERS

In [3]:
customers.head()

,CustomerId,CountryISOCode,BirthDate,Gender,FirstOrderDate
0,17860216.0,GB,22/10/2015 15:00,Femme,17/10/2015 12:58
1,10588204.0,FR,20/10/2015 19:46,Femme,20/10/2015 00:07
2,17841489.0,DE,20/10/2015 16:40,Homme,12/10/2015 23:04
3,17849688.0,DE,15/10/2015 14:17,Homme,22/10/2015 10:41
4,12916644.0,FR,13/10/2015 15:54,Femme,15/03/2012 10:04


### BirthDate -> Age

In [4]:
customers['BirthDate'] = customers['BirthDate'].apply(lambda x : datetime.strptime(x, '%d/%m/%Y %H:%M') )
customers['Age'] = 2018 - customers['BirthDate'].apply(lambda x : x.year)

In [5]:
for i in range(10,60,5):
    customers['Age' + str(i) + '-' + str(i+5)] = ((customers['Age']>i) & (customers['Age']<=i+5)).astype('float')

In [6]:
customers.head()

,CustomerId,CountryISOCode,BirthDate,Gender,FirstOrderDate,Age,Age10-15,Age15-20,Age20-25,Age25-30,Age30-35,Age35-40,Age40-45,Age45-50,Age50-55,Age55-60
0,17860216.0,GB,2015-10-22 15:00:00,Femme,17/10/2015 12:58,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10588204.0,FR,2015-10-20 19:46:00,Femme,20/10/2015 00:07,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,17841489.0,DE,2015-10-20 16:40:00,Homme,12/10/2015 23:04,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17849688.0,DE,2015-10-15 14:17:00,Homme,22/10/2015 10:41,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12916644.0,FR,2015-10-13 15:54:00,Femme,15/03/2012 10:04,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Gender -> Binaire

In [7]:
## 1 si une femme
customers['Sexe'] = customers['Gender'].eq('Femme').astype('float')

In [8]:
customers.head()

,CustomerId,CountryISOCode,BirthDate,Gender,FirstOrderDate,Age,Age10-15,Age15-20,Age20-25,Age25-30,Age30-35,Age35-40,Age40-45,Age45-50,Age50-55,Age55-60,Sexe
0,17860216.0,GB,2015-10-22 15:00:00,Femme,17/10/2015 12:58,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,10588204.0,FR,2015-10-20 19:46:00,Femme,20/10/2015 00:07,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,17841489.0,DE,2015-10-20 16:40:00,Homme,12/10/2015 23:04,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17849688.0,DE,2015-10-15 14:17:00,Homme,22/10/2015 10:41,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12916644.0,FR,2015-10-13 15:54:00,Femme,15/03/2012 10:04,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### FirstOrderDate -> Time in Year since first Order

In [9]:
customers['FirstOrderDate'] = customers['FirstOrderDate'].apply(lambda x : datetime.strptime(x, '%d/%m/%Y %H:%M') )
customers['FirstOrder'] = 2018 - customers['FirstOrderDate'].apply(lambda x : x.year)

In [10]:
customers.FirstOrder.value_counts()

3     198766
4     182812
5     150411
6     125762
7      64291
8      31366
9      14789
10      6881
11      3962
12      1164
13        34
Name: FirstOrder, dtype: int64

In [11]:
for i in range(3,10):
    customers['FirstOrder' + '-' + str(i) + 'Y'] = (customers['FirstOrder']==i).astype('float')
customers['FirstOrder-10Y+'] = (customers['FirstOrder']>=10).astype('float')    

In [12]:
customers.head()

,CustomerId,CountryISOCode,BirthDate,Gender,FirstOrderDate,Age,Age10-15,Age15-20,Age20-25,Age25-30,...,Sexe,FirstOrder,FirstOrder-3Y,FirstOrder-4Y,FirstOrder-5Y,FirstOrder-6Y,FirstOrder-7Y,FirstOrder-8Y,FirstOrder-9Y,FirstOrder-10Y+
0,17860216.0,GB,2015-10-22 15:00:00,Femme,2015-10-17 12:58:00,3,0.0,0.0,0.0,0.0,...,1.0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10588204.0,FR,2015-10-20 19:46:00,Femme,2015-10-20 00:07:00,3,0.0,0.0,0.0,0.0,...,1.0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,17841489.0,DE,2015-10-20 16:40:00,Homme,2015-10-12 23:04:00,3,0.0,0.0,0.0,0.0,...,0.0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17849688.0,DE,2015-10-15 14:17:00,Homme,2015-10-22 10:41:00,3,0.0,0.0,0.0,0.0,...,0.0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12916644.0,FR,2015-10-13 15:54:00,Femme,2012-03-15 10:04:00,3,0.0,0.0,0.0,0.0,...,1.0,6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


# ORDERS

In [13]:
# rename
orders = X_train

In [14]:
orders.head()

,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel,PayementModeLabel,CustomerTypeLabel,IsoCode,DeviceTypeLabel,PricingTypeLabel,TotalLineItems,Quantity,UnitPMPEUR,OrderCreationDate,OrderShipDate,OrderNumCustomer,IsOnSale,BillingPostalCode
0,73521754,439729,1,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,ND,Plein Tarif,2,1,"52,64",2011-10-26 12:10:48,2011-10-26 18:27:00,1,0.0,87000
1,73521754,440174,2,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,ND,Plein Tarif,2,1,"52,64",2011-10-26 12:10:48,2011-10-26 18:27:00,1,0.0,87000
2,73525226,494501,1,12443958,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,ND,Plein Tarif,1,1,"13,17",2011-10-26 12:11:38,2011-10-26 17:48:00,1,0.0,77700
3,73529009,439590,1,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,ND,Plein Tarif,2,1,"56,4",2011-10-26 12:13:09,2011-10-26 17:59:00,1,0.0,44600
4,73529009,559476,2,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,ND,Plein Tarif,2,1,37,2011-10-26 12:13:09,2011-10-26 17:59:00,1,0.0,44600


### LineItem -> LineItem binary position

In [15]:
#peut etre faire position 1 ou autre ?
for i in range(1,5):
    orders['LineItem' + '_' + str(i) ] = (orders['LineItem']==i).astype('float')
orders['LineItem_5+'] = (orders['LineItem']>=5).astype('float')    

In [16]:
orders.head()

,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel,PayementModeLabel,CustomerTypeLabel,IsoCode,...,OrderCreationDate,OrderShipDate,OrderNumCustomer,IsOnSale,BillingPostalCode,LineItem_1,LineItem_2,LineItem_3,LineItem_4,LineItem_5+
0,73521754,439729,1,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,2011-10-26 12:10:48,2011-10-26 18:27:00,1,0.0,87000,1.0,0.0,0.0,0.0,0.0
1,73521754,440174,2,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,2011-10-26 12:10:48,2011-10-26 18:27:00,1,0.0,87000,0.0,1.0,0.0,0.0,0.0
2,73525226,494501,1,12443958,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,2011-10-26 12:11:38,2011-10-26 17:48:00,1,0.0,77700,1.0,0.0,0.0,0.0,0.0
3,73529009,439590,1,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,2011-10-26 12:13:09,2011-10-26 17:59:00,1,0.0,44600,1.0,0.0,0.0,0.0,0.0
4,73529009,559476,2,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,2011-10-26 12:13:09,2011-10-26 17:59:00,1,0.0,44600,0.0,1.0,0.0,0.0,0.0


### OrderStatusLabel -> Binary (Expedie = 1)

In [17]:
orders['OrderStatus'] = orders['OrderStatusLabel'].eq('Expédié').astype('float')

In [18]:
orders.head()

,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel,PayementModeLabel,CustomerTypeLabel,IsoCode,...,OrderShipDate,OrderNumCustomer,IsOnSale,BillingPostalCode,LineItem_1,LineItem_2,LineItem_3,LineItem_4,LineItem_5+,OrderStatus
0,73521754,439729,1,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,2011-10-26 18:27:00,1,0.0,87000,1.0,0.0,0.0,0.0,0.0,1.0
1,73521754,440174,2,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,2011-10-26 18:27:00,1,0.0,87000,0.0,1.0,0.0,0.0,0.0,1.0
2,73525226,494501,1,12443958,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,2011-10-26 17:48:00,1,0.0,77700,1.0,0.0,0.0,0.0,0.0,1.0
3,73529009,439590,1,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,2011-10-26 17:59:00,1,0.0,44600,1.0,0.0,0.0,0.0,0.0,1.0
4,73529009,559476,2,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,2011-10-26 17:59:00,1,0.0,44600,0.0,1.0,0.0,0.0,0.0,1.0


### OrderTypelabel -> Binary (DIRECT = 1)

In [19]:
orders['OrderType'] = orders['OrderTypelabel'].eq('DIRECT').astype('float')

In [20]:
orders.head()

,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel,PayementModeLabel,CustomerTypeLabel,IsoCode,...,OrderNumCustomer,IsOnSale,BillingPostalCode,LineItem_1,LineItem_2,LineItem_3,LineItem_4,LineItem_5+,OrderStatus,OrderType
0,73521754,439729,1,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,1,0.0,87000,1.0,0.0,0.0,0.0,0.0,1.0,1.0
1,73521754,440174,2,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,1,0.0,87000,0.0,1.0,0.0,0.0,0.0,1.0,1.0
2,73525226,494501,1,12443958,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,1,0.0,77700,1.0,0.0,0.0,0.0,0.0,1.0,1.0
3,73529009,439590,1,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,1,0.0,44600,1.0,0.0,0.0,0.0,0.0,1.0,1.0
4,73529009,559476,2,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,1,0.0,44600,0.0,1.0,0.0,0.0,0.0,1.0,1.0


### SeasonLabel -> Binary (Automne/Hiver = 1)

In [21]:
orders['Season'] = orders['SeasonLabel'].eq('Automne/Hiver').astype('float')

In [22]:
orders.head()

,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel,PayementModeLabel,CustomerTypeLabel,IsoCode,...,IsOnSale,BillingPostalCode,LineItem_1,LineItem_2,LineItem_3,LineItem_4,LineItem_5+,OrderStatus,OrderType,Season
0,73521754,439729,1,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,87000,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
1,73521754,440174,2,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,87000,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0
2,73525226,494501,1,12443958,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,77700,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
3,73529009,439590,1,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,44600,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
4,73529009,559476,2,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,44600,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0


### PayementModeLabel -> Binary Payement Mode

In [23]:
orders.PayementModeLabel.value_counts()

Carte bancaire       816469
PayPal               154745
Gratuit               35241
iDeal                 22887
Chèque                17313
BankTransfer_DE        7039
DotPay                 5173
maestro                3550
Cod_DHL                3386
BankTransfer_IBAN      1487
Name: PayementModeLabel, dtype: int64

In [24]:
orders['Payement_CB'] = orders['PayementModeLabel'].eq('Carte bancaire').astype('float')
orders['Payement_PP'] = orders['PayementModeLabel'].eq('PayPal').astype('float')
orders['Payement_iDeal'] = orders['PayementModeLabel'].eq('iDeal').astype('float')
orders['Payement_Gratuit'] = orders['PayementModeLabel'].eq('Gratuit').astype('float')

In [25]:
orders.head()

,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel,PayementModeLabel,CustomerTypeLabel,IsoCode,...,LineItem_3,LineItem_4,LineItem_5+,OrderStatus,OrderType,Season,Payement_CB,Payement_PP,Payement_iDeal,Payement_Gratuit
0,73521754,439729,1,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
1,73521754,440174,2,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
2,73525226,494501,1,12443958,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
3,73529009,439590,1,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
4,73529009,559476,2,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0


### CustomerTypeLabel -> Binary Type (Fidelise = 1)

In [26]:
orders.CustomerTypeLabel.value_counts()

Fidélisé    615726
Nouveau     451564
Name: CustomerTypeLabel, dtype: int64

In [27]:
orders['CustomerType'] = orders['CustomerTypeLabel'].eq('Fidélisé').astype('float')

In [28]:
orders.head()

,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel,PayementModeLabel,CustomerTypeLabel,IsoCode,...,LineItem_4,LineItem_5+,OrderStatus,OrderType,Season,Payement_CB,Payement_PP,Payement_iDeal,Payement_Gratuit,CustomerType
0,73521754,439729,1,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,73521754,440174,2,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
2,73525226,494501,1,12443958,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
3,73529009,439590,1,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
4,73529009,559476,2,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


### IsoCode inchangé

### DeviceTypeLabel ne sera pas utilisé (ND majoritaire)

In [29]:
orders.DeviceTypeLabel.value_counts()

ND         877457
Desktop    157382
Tablet      23132
Mobile       9319
Name: DeviceTypeLabel, dtype: int64

### PricingTypeLabel -> Binary Type 

In [30]:
orders.PricingTypeLabel.value_counts()

Plein Tarif      450986
Soldes           318489
Promo Avec CP    144922
Promo Sans CP     95377
Vente Privée      57516
Name: PricingTypeLabel, dtype: int64

In [31]:
orders['Pricing_Plein'] = orders['PricingTypeLabel'].eq('Plein Tarif').astype('float')
orders['Pricing_Soldes'] = orders['PricingTypeLabel'].eq('Soldes').astype('float')
orders['Pricing_PromoCP'] = orders['PricingTypeLabel'].eq('Promo Avec CP').astype('float')
orders['Pricing_PromoPasCP'] = orders['PricingTypeLabel'].eq('Promo Sans CP').astype('float')
orders['Pricing_VentePrivée'] = orders['PricingTypeLabel'].eq('Vente Privée').astype('float')

In [32]:
orders.head()

,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel,PayementModeLabel,CustomerTypeLabel,IsoCode,...,Payement_CB,Payement_PP,Payement_iDeal,Payement_Gratuit,CustomerType,Pricing_Plein,Pricing_Soldes,Pricing_PromoCP,Pricing_PromoPasCP,Pricing_VentePrivée
0,73521754,439729,1,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,73521754,440174,2,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,73525226,494501,1,12443958,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,73529009,439590,1,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,73529009,559476,2,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


### TotalLineItems -> Same as LineItem

In [33]:
for i in range(1,6):
    orders['TotalLineItems' + '_' + str(i) ] = (orders['TotalLineItems']==i).astype('float')
orders['TotalLineItems_6+'] = (orders['TotalLineItems']>=6).astype('float')

In [34]:
orders.head()

,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel,PayementModeLabel,CustomerTypeLabel,IsoCode,...,Pricing_Soldes,Pricing_PromoCP,Pricing_PromoPasCP,Pricing_VentePrivée,TotalLineItems_1,TotalLineItems_2,TotalLineItems_3,TotalLineItems_4,TotalLineItems_5,TotalLineItems_6+
0,73521754,439729,1,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,73521754,440174,2,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,73525226,494501,1,12443958,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,73529009,439590,1,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,73529009,559476,2,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


### OrderCreationDate -> Categories based on order creation hour

In [ ]:
orders['OrderCreationDate'] = orders['OrderCreationDate'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )

In [46]:
temp = orders['OrderCreationDate'].apply(lambda x : x.hour)

orders['Hour_21H_1H'] = ((temp<=1) | (temp>=21)).astype('float')

In [48]:
orders.head()

,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel,PayementModeLabel,CustomerTypeLabel,IsoCode,...,Pricing_PromoCP,Pricing_PromoPasCP,Pricing_VentePrivée,TotalLineItems_1,TotalLineItems_2,TotalLineItems_3,TotalLineItems_4,TotalLineItems_5,TotalLineItems_6+,Hour_21H_1H
0,73521754,439729,1,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,73521754,440174,2,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,73525226,494501,1,12443958,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,73529009,439590,1,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,73529009,559476,2,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### OrderShipDate -> on pense que la date d'envoi ne joue pas dans le retour du produit